In [1]:
state = 'AR' 
year = 2020
obj_type = 'cut_edges'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of AR has 4 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from AR_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 4993 plans from file.


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 745353 and U = 760409 and k = 4.

****************************************
Trying deviation = 7528.81
****************************************
Using user-provided warm starts.
Selected warm_start = [[4, 7, 17, 20, 30, 31, 50, 51, 52, 53, 61, 71], [2, 3, 5, 10, 12, 13, 14, 15, 16, 26, 32, 38, 39, 41, 43, 44, 46, 49, 58, 59, 62, 63, 68, 69, 73, 74], [6, 8, 9, 11, 18, 19, 21, 24, 25, 27, 28, 29, 33, 34, 35, 36, 40, 42, 45, 47, 48, 54, 55, 56, 57, 60, 64, 65, 66, 67, 72], [0, 1, 22, 23, 37, 70]]
Objective value: 32
Deviation: 4022.0

****************************************
Running labeling model!
****************************************
L = 745353 and U = 760409
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Applying warm s

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.43651    0  372   33.00000    9.43651  71.4%     -    0s
     0     0   10.08196    0  402   33.00000   10.08196  69.4%     -    0s
     0     0   10.11654    0  404   33.00000   10.11654  69.3%     -    0s
     0     0   10.11654    0  404   33.00000   10.11654  69.3%     -    0s
     0     0   10.51003    0  409   33.00000   10.51003  68.2%     -    0s
     0     0   10.52715    0  414   33.00000   10.52715  68.1%     -    0s
     0     0   10.52715    0  414   33.00000   10.52715  68.1%     -    1s
     0     0   10.55735    0  414   33.00000   10.55735  68.0%     -    1s
     0     0   10.58016    0  413   33.00000   10.58016  67.9%     -    1s
     0     0   10.81231    0  421   33.00000   10.81231  67.2%     -    1s
     0     0   10.83248    0  422   33.00000   10.83248  67.2%     -    1s
     0     0   10.83367    0  420   33.00000   10.83367  67.2%     -    1s
     0     0   10.8342

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
no_solution_region = [0, min(round(r[2],1) for r in result)]
print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)